In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.integrate import trapezoid, solve_ivp

In [ ]:
num_pontos = 200
Massa = 10                                                      # massa de 10Kg

CampoMagnetico = np.array([0, 68, 135, 203, 271, 338, 406, 474, 542, 609, 1100, 1500, 2500, 4000, 5000, 9000, 12000, 20000, 25000])
DensidadeCampo = np.array([0, 0.733, 1.205, 1.424, 1.517, 1.560, 1.588, 1.617, 1.631, 1.646, 1.689, 1.703,1.724, 1.731, 1.738, 1.761, 1.770, 1.800, 1.816])

In [ ]:
def Coenergia_forca(Interpolacao_H_B, Nf):
    # variação de x
    x = np.linspace(0, 0.02, num_pontos)# 20 mm em metros

    # variação de Bm
    Bm = np.linspace(0, 1.2, num_pontos)
    # Corrente ficticia -> Inicia um vetor com todos os valores nulos
    If = np.zeros(num_pontos)
    # Fluxo concatenado
    fluxo_concatenado = np.zeros(num_pontos)
    Coenergia = np.zeros(num_pontos)

    #caso onde não calculamos por bobina ficticia
    CoenergiaN1000 = np.zeros(num_pontos)

    for j in range(len(x)):
      for b in range(len(Bm)):

        # If = (Hi*d + Bm*[x + g0]/U0) /Nf
        If[b] = ((Interpolacao_H_B(Bm[b])*d) + ((Bm[b]*(x[j]+g0)/U0)))/Nf

        # λ =  Nf * Bm * Area_do_imã
        fluxo_concatenado[b] =  Nf*Bm[b]*Area

        if (f"{x[j]:.4f}" == "0.0005"):  # Posição do elemento móvel
            # Encontrando os valores da densidade de fluxo B para a corrente If
            Bm_x = np.copy(Bm)
            If_x = np.copy(If)

      # Interpolação de λ , Bm, If
      interpolacao_fluxo_concatenado_If = CubicSpline(If,fluxo_concatenado)
      interpolacao_If_Bm = CubicSpline(Bm,If)

      #parte inferior da integral, ponto onde B = 0
      If_0 = interpolacao_If_Bm(0)
      #limite de integração do λ
      lim_int_fluxo_concatenado = np.linspace(If_0, 0, num_pontos)
      #valores para coenergia
      interpol_fluxo_concatenado = interpolacao_fluxo_concatenado_If(lim_int_fluxo_concatenado)
      #valores para coenergia
      Coenergia[j] = trapezoid(interpol_fluxo_concatenado,lim_int_fluxo_concatenado) 
      
      Bop_normal = 1.08
      Hop_normal = -31.14*1e+3
      xop_normal = 0.5*1e-3
      #valor da corrente quando está em operação normal
      Iop_normal =   ((-Hop_normal*d) + Bop_normal*((xop_normal + g0)/U0))/Nf
      
      Iint_Nf_1000 = np.linspace(If_0,Iop_normal,num_pontos)
      Fluxo_concat_Interpolado_Nf_1000 = interpolacao_fluxo_concatenado_If(Iint_Nf_1000)
      CoenergiaN1000[j] = trapezoid(Fluxo_concat_Interpolado_Nf_1000,Iint_Nf_1000)

    #caso de operação normal  
    if(Nf == 1000):
      Forca = np.gradient(CoenergiaN1000[:len(x)-1], x[:len(x)-1])
      return Forca, CoenergiaN1000[:len(CoenergiaN1000)-1],(x[:len(x)-1]), Bm_x, If_x, Iop_normal 

    #caso da bobina ficticia
    else:
      Forca = np.gradient(Coenergia[:len(x)-1], x[:len(x)-1])
      return Forca, Coenergia[:len(Coenergia)-1],(x[:len(x)-1])